In [7]:
import nbimporter
from U_Value_to_Detail_Specification import convert as convertUValueToSpec

Importing Jupyter notebook from U_Value_to_Detail_Specification.ipynb


# 外皮情報LV1からLV2へのコンバート

## 1. 外皮簡易法＋用途別床面積
- 当該住戸の外皮の部位の面積等を用いずに簡易熱負荷計算を実施する場合には、部位の面積および長さを、「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）」２．エネルギー消費性能の算定方法　2.2　算定方法　第三章　 暖冷房負荷と外皮性能　	第二節　外皮性能　9.当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法　に示された標準住戸（以下、「簡易外皮入力モデル」）における部位の面積および長さ等より推定する。
- 屋根・天井、外壁、床の構成は、入力値である部位種別の熱貫流率をもとに、部位のU値から壁体構成を決定する方法により決定する。
- 土間床等の外周部の線熱貫流率については、簡易熱負荷計算での扱いに従う
- 開口部の日射熱取得率および外部日除けの取得日射熱補正係数は、それぞれ、季節別に、垂直面日射熱取得率および取得日射熱補正係数の入力値もしくは規定値を使用する。この場合、簡易熱負荷計算においては、開口部の入射角特性および外部日除けは考慮しない。
- その他は、外皮性能計算プログラムの入力情報に用途別床面積を追加したものと同様に求める。

#### 《設計住戸の各部位の長さおよび面積の推定》
$$ \qquad
A_{i,d}= A_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (1)
\\
$$
$$ \qquad
L_{i,d}= L_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (2)
\\
$$

- 入力値：

  - 簡易外皮入力モデル住戸の延床面積$[m ^ 2]$：$A_{floor,s}$
  - 設計住戸の延床面積$[m ^ 2]$：$A_{floor,d}$
  - 簡易外皮入力モデル住戸の部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,s}$
  - 簡易外皮入力モデル住戸の部位（土間床等の外周部）𝑖の長さ$[m]$：$L_{i,s}$

- 出力値：

  - 設計住戸の部位（一般部位又は開口部）𝑖の面積の推定値$[m ^ 2]$：$A_{i,d}$
  - 設計住戸の部位（土間床等の外周部）𝑖の長さの推定値$[m]$：$L_{i,d}$

#### Function

In [8]:
def GetDesignPartsArea(parts_area_standard, total_floor_area_design):
    TOTAL_FLOOR_AREA_STANDARD = 90.0  #簡易外皮入力モデル住戸の延床面積(m2)
    partsArea = round(parts_area_standard * total_floor_area_design / TOTAL_FLOOR_AREA_STANDARD,2)
    return partsArea

#### Example

In [9]:
GetDesignPartsArea(50.85,120)

67.8

## 2. コンバート

### 1) 共通

- 地域の区分、主たる居室の床面積(m<sup>2</sup>)、その他の床面積(m<sup>2</sup>)、床面積の合計(m<sup>2</sup>)の情報は維持する。
- 新たに変数「簡易外皮入力フラグ」を作成し、LV1から生成したLV2情報は必ずTrueとする。

#### Input

```
{
    'Region'                     : # 地域の区分 1 ~ 8 の整数
    'MainOccupantRoomFloorArea'  : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea' : # その他の居室の床面積(m2)
    'TotalFloorArea'             : # 床面積の合計(m2)
}
```

#### Output

```
{
    'Region'                      : # 地域の区分 1 ~ 8 の整数
    'IsSimplifiedInput'           : # bool値 簡易入力から生成したデータか否かを表す, LV1からの場合は常にTrueとなる。
    'MainOccupantRoomFloorArea'   : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea'  : # その他の居室の床面積(m2)
    'TotalFloorArea'              : # 床面積の合計(m2)
}
```

#### Function

In [10]:
def convert_common(d):
    return {
        'Region'                     : d['Region'],
        'IsSimplifiedInput'          : True,
        'MainOccupantRoomFloorArea'  : d['MainOccupantRoomFloorArea'],
        'OtherOccupantRoomFloorArea' : d['OtherOccupantRoomFloorArea'],
        'TotalFloorArea'             : d['TotalFloorArea']
    }

#### Example

In [11]:
convert_common({
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0
})

{'IsSimplifiedInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloorArea': 60.0,
 'Region': 6,
 'TotalFloorArea': 120.0}

### 2) 開口部以外の不透明部位

- 部位のU値から壁体構成を決定する

#### 簡易外皮入力モデルにおける開口部以外の不透明部位の面積

- 簡易外皮入力モデルにおける開口部以外の不透明部位の面積は、壁体種別、方位および断熱構造から決定する。

In [12]:
def getStandardPartsArea():
    return {
        'ceiling': {
            'top': {
                'floor_ins': 50.85,
                'base_ins': 50.85
            } 
        } ,
        'wall': {
            'SW': {
                'floor_ins': 30.47,
                'base_ins': 30.47
            } ,
            'NW': {
                'floor_ins': 22.37,
                'base_ins': 22.37
            } ,
            'NE': {
                'floor_ins': 47.92,
                'base_ins':47.92
            } ,
            'SE': {
                'floor_ins': 22.28,
                'base_ins':22.28
            } 
        } ,
        'floor' : {
            'bottom' : {
                'floor_ins': 45.05,
                'base_ins':0.0
            }
        }
    }    

##### Example

In [13]:
getStandardPartsArea()['wall']['SW']['floor_ins']

30.47

#### Input

```
{
    'InsulationType'                  : # 断熱構造 string値,  'floor_ins' or 'base_ins'
    'U_roof'                          : # 屋根・天井の熱貫流率(W/m2K)
    'U_wall'                          : # 外壁の熱貫流率(W/m2K)
    'U_floor'                         : # 床の熱貫流率(W/m2K)
    'U_door'                          : # ドアの熱貫流率(W/m2K)
    'IsInputEarthFloorPerimeterPsi' : # 土間床等の外周部の線熱貫流率を入力するかどうか, bool値
    'Psi_base_others'                 : # その他の土間床等の外周部の線熱貫流率(W/mK),
                                          'IsInputEarthFloorPerimeterPsi'がFalseの場合は無視される。
    'Psi_base_entrance'               : # 玄関等の土間床等の外周部の線熱貫流率(W/mK)
                                          'IsInputEarthFloorPerimeterPsi'がFalseの場合は無視される。
}
```

#### Output

```
{
    'Name'                        : # 壁体名称
    'Structure'                   : # 一般部位構造種別 1 ~ 4 の整数
    'Type'                        : # 壁体種別 1 ~ 6 の整数
    'Direction'                   : # 方位　string値, 以下のいずれか
                                    # 'top'/'N'/'NE'/'E'/'SE'/'S'/'SW'/'W'/'NW'/'bottom'/
                                    # 'ClosedSpace'（外気に通じていない空間）/
                                    # 'OpenBackFloor'（外気に通じる床裏）/
                                    # 'ResidenceSpace'（住戸及び住戸と同様の熱的環境の空間）/
                                    # 'ClosedBackFloor'（外気に通じていない床裏）/
                                    # 'EarthFloor'（土間床等）
    'Area'                        : # 部位面積(m2)
    'parts' : [
        {
            layers : [
                {
                    'Name'   : # 部材名称　string値
                    'Thick'  : # 厚さ(m)
                    'Cond'   : # 熱伝導率(W/(m・K))
                    'Spech'  : # 容積比熱(J/(m3・K))
                }
            ] # 層構成の情報
        ] # 部位の情報
}
```

### Function

In [14]:
def convert_wall( d ):
    
    #簡易外皮入力モデルにおける開口部以外の不透明部位のリストを作成
    standard_parts = [
        { 'name': 'Ceiling', 'type': 'ceiling', 'structure': 'wood', 'direction': 'top',    'Ua': d['U_roof']  },
        { 'name': 'Wall_SW', 'type': 'wall',    'structure': 'wood', 'direction': 'SW',     'Ua': d['U_wall']  },
        { 'name': 'Wall_NW', 'type': 'wall',    'structure': 'wood', 'direction': 'NW',     'Ua': d['U_wall']  },
        { 'name': 'Wall_NE', 'type': 'wall',    'structure': 'wood', 'direction': 'NE',     'Ua': d['U_wall']  },
        { 'name': 'Wall_SE', 'type': 'wall',    'structure': 'wood', 'direction': 'SE',     'Ua': d['U_wall']  },
        { 'name': 'Floor',   'type': 'floor',   'structure': 'wood', 'direction': 'bottom', 'Ua': d['U_floor'] }
    ]
                  
    walls = []
    for x in standard_parts:
        
        #標準住戸の部位の面積を取得
        partsArea = getStandardPartsArea()[ x['type'] ][ x['direction'] ][ d['InsulationType'] ]

        #層構成を取得
        layers = convertUValueToSpec( x['structure'], x['type'], x['Ua'] )
        
        part = {
            'AreaRatio': 1.0,
            'Layers'   : [ { 'Name' : l.name, 'Cond' : l.cond, 'Thick' : l.thick, 'Spech' : l.spech } for l in layers ]
        }
        
        wall ={
            'Name': x['name'],
            'Type': x['type'],
            'Structure': x['structure'],
            'Direction': x['direction'],
            'Area': GetDesignPartsArea( partsArea, d['TotalFloorArea'] ),
            'Parts':[part]
        }

        walls.append(wall)
   
    return { 'Walls' : walls }

### Example

In [15]:
d = {
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0,
    'InsulationType' : 'floor_ins',
    'U_roof' : 7.7,
    'U_wall' : 6.67,
    'U_floor' : 5.27,
    'U_door' : 4.65,
    'IsInputEarthFloorPerimeterPsi' : True,
    'Psi_base_others' : 1.8,
    'Psi_base_entrance' :1.8
}

convert_wall( d )

{'Walls': [{'Area': 67.8,
   'Direction': 'top',
   'Name': 'Ceiling',
   'Parts': [{'AreaRatio': 1.0,
     'Layers': [{'Cond': 0.045, 'Name': 'GW16K', 'Spech': 13.0, 'Thick': 0.0},
      {'Cond': 0.22, 'Name': 'GPB', 'Spech': 830.0, 'Thick': 0.0095}]}],
   'Structure': 'wood',
   'Type': 'ceiling'},
  {'Area': 40.63,
   'Direction': 'SW',
   'Name': 'Wall_SW',
   'Parts': [{'AreaRatio': 1.0,
     'Layers': [{'Cond': 0.045, 'Name': 'GW16K', 'Spech': 13.0, 'Thick': 0.0},
      {'Cond': 0.22, 'Name': 'GPB', 'Spech': 830.0, 'Thick': 0.0095}]}],
   'Structure': 'wood',
   'Type': 'wall'},
  {'Area': 29.83,
   'Direction': 'NW',
   'Name': 'Wall_NW',
   'Parts': [{'AreaRatio': 1.0,
     'Layers': [{'Cond': 0.045, 'Name': 'GW16K', 'Spech': 13.0, 'Thick': 0.0},
      {'Cond': 0.22, 'Name': 'GPB', 'Spech': 830.0, 'Thick': 0.0095}]}],
   'Structure': 'wood',
   'Type': 'wall'},
  {'Area': 63.89,
   'Direction': 'NE',
   'Name': 'Wall_NE',
   'Parts': [{'AreaRatio': 1.0,
     'Layers': [{'Cond':

### 3) 開口部の透明部位

- 開口部名称、日射熱取得率[-]、熱貫流率[W/(m<sup>2</sup>・K)]の情報は維持する。

#### 簡易外皮入力モデルにおける開口部の透明部位の面積

- 簡易外皮入力モデルにおける開口部の透明部位の面積は、方位から決定する。

In [16]:
def getStandardPartsAreaWindow():
    return {
        'SW': 22.69,
        'NW': 2.38,
        'NE': 3.63,
        'SE': 4.37
    }

##### Example

In [17]:
getStandardPartsAreaWindow()['SW']

22.69

#### 日除けの出巾を計算する関数

- 取得日射熱補正係数、地域区分、方位、開口部高さ、窓上端と外部日除けの距離から出巾を計算する関数を定義する。
- 取得日射熱補正係数を求める(3)式を出巾$z$について求める。(3)式中の係数$a,b,c,d$は、地域区分及び方位により変化する。

$$ \qquad
f = a×(b + c×\frac{d×y_{1}+y_{2}}{z})
\qquad (3)
\\
$$

  - 取得日射熱補正係数$[-]$：$f$
  - 日除け下端から窓上端までの垂直方向の距離$[m]$：$y_{1}$
  - 窓の開口高さ$[m]$：$y_{2}$
  - 壁面からの日除けの出巾$[m]$：$z$

In [18]:
def getOverhangWidth(isHeating, Region, direction, distance, height, f ):
    # isHeating as Boolean
    # Region as integer: 1 to 8
    # direction as string: top, N, NE, E, SE, S, SW, W, NW, bottom, ClosedSpace, OpenBackFloor, ResidenceSpace, ClosedBackFloor, EarthFloor
    # distance: vertical distance from bottom of sunshade to top of window (m)
    # height: window height (m)
    # f: solar heat correction factor (-)　取得日射熱補正係数
    
    y1 = distance
    y2 = height
    
    # 取得日射熱補正係数を調整
    if isHeating == True:
        if f > 0.72:
            f = 0.72
    else:
        if f > 0.93:
            f = 0.93
    
    # 係数a,b,c,dを設定
    if isHeating == True:  # 暖房期
        if Region <= 7:
            if ( direction == 'SE' or direction == 'S' or direction == 'SW' ):
                a, b, c, d = (0.01, 5, 20, 3)
            else:
                a, b, c, d = (0.01, 10, 15, 2)
        elif Region == 8 :
            pass #8地域の場合はどうする？
        
    else:  # 冷房期
        if Region <= 7:
            if ( direction == 'S' ):
                a, b, c, d = (0.01, 24, 9, 3)
            else:
                a, b, c, d = (0.01, 16, 24, 2)
        elif Region == 8 :
            if ( direction == 'SE' or direction == 'S' or direction == 'SW' ):
                a, b, c, d = (0.01, 16, 19, 2)
            else:
                a, b, c, d = (0.01, 16, 24, 2)
    
    # 出巾を計算
    z = ( c * ( d * y1 + y2) ) / ( ( f / a ) - b)
    
    return z


##### Example

In [19]:
getOverhangWidth(True, 6, 'NE', 0, 1.1, 0.72)

0.2661290322580645

#### Input

```
data = {
    'U_wnd'           : # 窓の熱貫流率 (W/m2K)
    'eta_d_C'         : # 垂直面日射熱取得率(夏期) (-)
    'eta_d_H'         : # 垂直面日射熱取得率（冬期） (-)
    'IsInputFValue'   : # 取得日射熱補正係数を入力するかどうか, bool値
    'f_value_C'       : # 取得日射熱補正係数(夏期)
    'f_value_H'       : # 取得日射熱補正係数(冬期)
}
```

#### Output

```
data = {
    'name'                        : # 開口部名称
    'direction'                   : # 方位　string値, 以下のいずれか
                                    # 'top'/'N'/'NE'/'E'/'SE'/'S'/'SW'/'W'/'NW'/'bottom'/
                                    # 'ClosedSpace'（外気に通じていない空間）/
                                    # 'OpenBackFloor'（外気に通じる床裏）/
                                    # 'ResidenceSpace'（住戸及び住戸と同様の熱的環境の空間）/
                                    # 'ClosedBackFloor'（外気に通じていない床裏）/
                                    # 'EarthFloor'（土間床等）
    'A'                           : # 部位面積(m2)
    'type'                        : # 窓の種類 string値, 'single' or 'double'
    'etaCooling'                  : # 日射熱取得率（夏期）(-)
    'etaHeating'                  : # 日射熱取得率（冬期）(-)
    'Uw'                          : # 窓の熱貫流率(W/(m2・K))
    'IsSunshadeInput'             : # 日除けを入力するかどうか, bool値
    'y2'                          : # 開口部高さ(m)
    'z'                           : # 出巾(m)
    'y1'                          : # 窓上端と外部日除けの距離(m)
}
```

### Function

In [10]:
def convert_window(d1):
    return {'name': d1['name'],
            'etac': d1['etac'],
            'etah': d1['etah'],
            'U': d1['U']}

### Example

In [11]:
convert_window({
    'name': 'Sample Window',
    'etac': 0.738,
    'etah': 0.738,
    'U': 4.65})

{'U': 4.65, 'etac': 0.738, 'etah': 0.738, 'name': 'Sample Window'}

## 2-4. コンバート【分類：外部日除け】

- 外部日除け名称、取得日射熱補正係数[-]の情報は維持する。

### Input

```
data = {
    'name'                        : # 外部日除け名称
    'fCooling'                    : # 取得日射熱補正係数（夏期）(-)
    'fHeating'                    : # 取得日射熱補正係数（冬期）(-)
}
```

### Output

```
data = {
    'name'                        : # 外部日除け名称
    'fCooling'                    : # 取得日射熱補正係数（夏期）(-)
    'fHeating'                    : # 取得日射熱補正係数（冬期）(-)
}
```

### Function

In [12]:
def convert_sunshade(d1):
    return {'name': d1['name'],
            'fc': d1['fc'],
            'fh': d1['fh']}

### Example

In [13]:
convert_sunshade({
    'name': 'Sample Sunshade',
    'fc': 0.700,
    'fh': 0.600,})

{'fc': 0.7, 'fh': 0.6, 'name': 'Sample Sunshade'}

## 2-5. コンバート【分類：部位情報】

- 簡易外皮入力モデル住戸の部位の長さおよび面積、設計住戸の延床面積より、設計住戸の各部位の長さおよび面積の推定を行う。

### Input

```
data = {
    'name'                        : # 壁体・開口部名称
    'FlagPerimeter'               : # 外皮フラグ 1 ~ 2 の整数
    'FlagSolarRadiation'          : # 日射フラグ 1 ~ 2 の整数
    'direction'                   : # 方位 1 ~ 14 の整数
    'FlagUnsteady'                : # 非定常フラグ 1 ~ 2 の整数
    'A'                           : # 部位面積(m2)
    'L'                           : # 部位長さ(m)
    'sunshade'                    : # 外部日除け名称
}
```

### Output

```
data = {
    'name'                        : # 壁体・開口部名称
    'FlagPerimeter'               : # 外皮フラグ 1 ~ 2 の整数
    'FlagSolarRadiation'          : # 日射フラグ 1 ~ 2 の整数
    'direction'                   : # 方位 1 ~ 14 の整数
    'FlagUnsteady'                : # 非定常フラグ 1 ~ 2 の整数
    'A'                           : # 部位面積(m2)
    'L'                           : # 部位長さ(m)
    'sunshade'                    : # 外部日除け名称
}
```

### Function

In [14]:
def convert_element(d1,d):
    FloorAreaS=90
    FloorArea=d['Common']['TotalFloorArea']
    return {'name': d1['name'],
            'FlagPerimeter': d1['FlagPerimeter'],
            'FlagSolarRadiation': d1['FlagSolarRadiation'],
            'FlagUnsteady': d1['FlagUnsteady'],
            'direction': d1['direction'],
            'AorL': round(d1['AorL']*FloorArea/FloorAreaS,2)}

### Example

In [15]:
convert_element({
    'name': 'Sample Element',
    'FlagPerimeter': 1,
    'FlagSolarRadiation': 1,
    'FlagUnsteady': 1,
    'direction': 1,
    'AorL': 1},{
    'Common':{
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0,}})

{'AorL': 1.33,
 'FlagPerimeter': 1,
 'FlagSolarRadiation': 1,
 'FlagUnsteady': 1,
 'direction': 1,
 'name': 'Sample Element'}

## 2-6. コンバート【分類：簡易外皮入力】

- 断熱構造、屋根・天井熱貫流率[W/(m<sup>2</sup>・K)]、外壁熱貫流率[W/(m<sup>2</sup>・K)]、床熱貫流率[W/(m<sup>2</sup>・K)]、ドア熱貫流率[W/(m<sup>2</sup>・K)]、垂直面日射熱取得率[-]、取得日射熱補正係数の入力[-]、取得日射熱補正係数[-]、土間床等の外周部の入力、一般線熱貫流率[W/(m・K)]、玄関等線熱貫流率[W/(m・K)]の情報は維持する。

### Input

```
data = {
    'InsulationStructure'        : # 床/基礎の断熱構造 1 ~ 2 の整数
    'UCeiling'                   : # 屋根・天井熱貫流率(W/(m2・K))
    'UWall'                      : # 外壁熱貫流率(W/(m2・K))
    'UFloor'                     : # 床熱貫流率(W/(m2・K))
    'UDoor'                      : # ドア熱貫流率(W/(m2・K))
    'UWindow'                    : # 窓熱貫流率(W/(m2・K))
    'etaWindowCooling'           : # 垂直面日射熱取得率（夏期）   
    'etaWindowHeating'           : # 垂直面日射熱取得率（冬期）
    'FlagfWinowInput'            : # 取得日射熱補正係数の入力の入力 1 ~ 2 の整数
    'fWindowCooling'             : # 取得日射熱補正係数（夏期）
    'fWindowHeating'             : # 取得日射熱補正係数（冬期）
    'FlagPerimeterInput'         : # 土間床等の外周部の入力 1 ~ 2 の整数
    'psiPerimeter'               : # 一般線熱貫流率(W/(m・K))
    'psiPerimeterDoor'           : # 玄関等線熱貫流率(W/(m・K))
}
```

### Output

```
data = {
    'InsulationStructure'        : # 床/基礎の断熱構造 1 ~ 2 の整数
    'UCeiling'                   : # 屋根・天井熱貫流率(W/(m2・K))
    'UWall'                      : # 外壁熱貫流率(W/(m2・K))
    'UFloor'                     : # 床熱貫流率(W/(m2・K))
    'UDoor'                      : # ドア熱貫流率(W/(m2・K))
    'UWindow'                    : # 窓熱貫流率(W/(m2・K))
    'etaWindowCooling'           : # 垂直面日射熱取得率（夏期）   
    'etaWindowHeating'           : # 垂直面日射熱取得率（冬期）
    'FlagfWinowInput'            : # 取得日射熱補正係数の入力の入力 1 ~ 2 の整数
    'fWindowCooling'             : # 取得日射熱補正係数（夏期）
    'fWindowHeating'             : # 取得日射熱補正係数（冬期）
    'FlagPerimeterInput'         : # 土間床等の外周部の入力 1 ~ 2 の整数
    'psiPerimeter'               : # 一般線熱貫流率(W/(m・K))
    'psiPerimeterDoor'           : # 玄関等線熱貫流率(W/(m・K))
}
```

### Function

In [16]:
def convert_SimplifiedEnvelopeInput(d): 
    
    if d['FlagPerimeterInput']==2:
        d['psiPerimeter']=1.8
        d['psiPerimeterDoor']=1.8
    
    return {
    'InsulationStructure'        : d['InsulationStructure'],
    'UCeiling'                   : d['UCeiling'],
    'UWall'                      : d['UWall'],
    'UFloor'                     : d['UFloor'],
    'UDoor'                      : d['UDoor'],
    'UWindow'                    : d['UWindow'],
    'etaWindowCooling'           : d['etaWindowCooling'],  
    'etaWindowHeating'           : d['etaWindowHeating'],
    'FlagfWinowInput'            : d['FlagfWinowInput'],
    'fWindowCooling'             : d['fWindowCooling'],
    'fWindowHeating'             : d['fWindowHeating'],
    'FlagPerimeterInput'         : d['FlagPerimeterInput'],
    'psiPerimeter'               : d['psiPerimeter'],
    'psiPerimeterDoor'           : d['psiPerimeterDoor']
    }

### Example

In [17]:
convert_SimplifiedEnvelopeInput({
    'InsulationStructure'        : 1,
    'UCeiling'                   : 0.24,
    'UWall'                      : 0.53,
    'UFloor'                     : 0.48,
    'UDoor'                      : 2.33,
    'UWindow'                    : 3.49,
    'etaWindowCooling'           : 0.51,  
    'etaWindowHeating'           : 0.51,
    'FlagfWinowInput'            : 1,
    'fWindowCooling'             : 0.70,
    'fWindowHeating'             : 0.60,
    'FlagPerimeterInput'         : 2,
    'psiPerimeter'               : 1.80,
    'psiPerimeterDoor'           : 1.80
})

{'FlagPerimeterInput': 2,
 'FlagfWinowInput': 1,
 'InsulationStructure': 1,
 'UCeiling': 0.24,
 'UDoor': 2.33,
 'UFloor': 0.48,
 'UWall': 0.53,
 'UWindow': 3.49,
 'etaWindowCooling': 0.51,
 'etaWindowHeating': 0.51,
 'fWindowCooling': 0.7,
 'fWindowHeating': 0.6,
 'psiPerimeter': 1.8,
 'psiPerimeterDoor': 1.8}

## 2-7. コンバート【全分類】

In [18]:
def Convert(d):
    d_common = convert_common(d['Common'])
    d_SimplifiedEnvelopeInput=convert_SimplifiedEnvelopeInput(d['SimplifiedEnvelopeInput'])

    d_ceiling=convert_wall( {'structure':1,'name': 'Ceiling','type': 1,'Ua': d_SimplifiedEnvelopeInput['UCeiling']})
    d_wall=convert_wall( {'structure':1,'name': 'Wall','type': 2,'Ua': d_SimplifiedEnvelopeInput['UWall']})
    d_floor=convert_wall( {'structure':1,'name': 'Floor','type': 3,'Ua': d_SimplifiedEnvelopeInput['UFloor']})
    d_window=convert_window({'name': 'Window','etac': d_SimplifiedEnvelopeInput['etaWindowCooling'],'etah': d_SimplifiedEnvelopeInput['etaWindowHeating'],'U': d_SimplifiedEnvelopeInput['UWindow']})
    d_door=convert_window({'name': 'Door','etac':round(d_SimplifiedEnvelopeInput['UDoor']*0.034,3),'etah':round(d_SimplifiedEnvelopeInput['UDoor']*0.034,3),'U': d_SimplifiedEnvelopeInput['UDoor']})
    d_sunshade=convert_sunshade({'name': 'Sunshade','fc': d_SimplifiedEnvelopeInput['fWindowCooling'],'fh': d_SimplifiedEnvelopeInput['fWindowHeating']})

    #番号,外皮フラグ[-],日射フラグ[-],方位[-],非定常フラグ[-],壁体・開口部名称[-],部位面積[㎡],外部日除け名称[-]
    #ElementNo,FlagPerimeter,FlagSolarRadiation,direction,FlagUnsteady,name,A,Sunshade
    #標準住戸における部位の面積及び土間床等の外周部の長さ等（い）床断熱住戸の場合
    if d_SimplifiedEnvelopeInput['InsulationStructure']==1:
        StandardHouseElemntsFI = [ {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 1,'FlagUnsteady':1,'name':'Ceiling', 'AorL':50.85,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'Wall',    'AorL':30.47,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'Wall',    'AorL':22.37,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'Wall',    'AorL':47.92,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'Wall',    'AorL':22.28,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':2,'name':'Door',    'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':2,'name':'Door',    'AorL': 1.89,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':2,'name':'Door',    'AorL': 1.62,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':2,'name':'Door',    'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':2,'name':'Window',  'AorL':22.69,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':2,'name':'Window',  'AorL': 2.38,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':2,'name':'Window',  'AorL': 3.63,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':2,'name':'Window',  'AorL': 4.37,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'Floor',   'AorL':45.05,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'AbaseSW', 'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'AbaseNW', 'AorL': 0.91,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'AbaseNE', 'AorL': 0.91,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'AbaseSE', 'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'AbaseIF', 'AorL': 1.82,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'AbasedSW','AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'AbasedNW','AorL': 0.33,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'AbasedNE','AorL': 0.25,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'AbasedSE','AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'AbasedIF','AorL': 0.57,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'LprmSW',  'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'LprmNW',  'AorL': 1.82,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'LprmNE',  'AorL': 1.82,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'LprmSE',  'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'LprmIF',  'AorL': 3.64,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'LprmdSW', 'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'LprmdNW', 'AorL': 1.82,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'LprmdNE', 'AorL': 1.37,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'prmdSE',  'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'LprmdIF', 'AorL': 3.19,'Sunshade':None} ]
    #標準住戸における部位の面積及び土間床等の外周部の長さ等（ろ）基礎断熱住戸の場合
    elif d_SimplifiedEnvelopeInput['InsulationStructure']==2:
        StandardHouseElemntsFI = [ {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 1,'FlagUnsteady':1,'name':'Ceiling', 'AorL':50.85,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'Wall',    'AorL':30.47,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'Wall',    'AorL':22.37,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'Wall',    'AorL':47.92,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'Wall',    'AorL':22.28,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':2,'name':'Door',    'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':2,'name':'Door',    'AorL': 1.89,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':2,'name':'Door',    'AorL': 1.62,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':2,'name':'Door',    'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':2,'name':'Window',  'AorL':22.69,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':2,'name':'Window',  'AorL': 2.38,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':2,'name':'Window',  'AorL': 3.63,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':2,'name':'Window',  'AorL': 4.37,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'Floor',   'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'AbaseSW', 'AorL': 5.30,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'AbaseNW', 'AorL': 1.48,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'AbaseNE', 'AorL': 4.62,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'AbaseSE', 'AorL': 2.40,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'AbaseIF', 'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'AbasedSW','AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'AbasedNW','AorL': 0.33,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'AbasedNE','AorL': 0.25,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'AbasedSE','AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'AbasedIF','AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'LprmSW',  'AorL':10.61,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'LprmNW',  'AorL': 2.97,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'LprmNE',  'AorL': 9.24,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'LprmSE',  'AorL': 4.79,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'LprmIF',  'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'LprmdSW', 'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'LprmdNW', 'AorL': 1.82,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'LprmdNE', 'AorL': 1.37,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'prmdSE',  'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'LprmdIF', 'AorL': 0.00,'Sunshade':None} ]

    dictionary_element=[]
    for e in StandardHouseElemntsFI:
        temporary=convert_element(e, d)
        if e['name']=='Ceiling':
            temporary['Wall']=d_ceiling
        elif e['name']=='Wall':
            temporary['Wall']=d_wall
        elif e['name']=='Floor':
            temporary['Wall']=d_floor
        elif e['name']=='Window':
            temporary['Window']=d_window
            temporary['Sunshade']=d_sunshade
        elif e['name']=='Door':
            temporary['Door']=d_door
        dictionary_element.append(temporary)

    return {'common':d_common,'element':dictionary_element}

In [19]:
d = {
    'Common' : {
        'Region'                     : 6,
        'MainOccupantRoomFloorArea'  : 30.0,
        'OtherOccupantRoomFloorArea' : 60.0,
        'TotalFloorArea'             : 120.0
    },
    'SimplifiedEnvelopeInput' : {
        'InsulationStructure'        : 1,
        'UCeiling'                   : 0.24,
        'UWall'                      : 0.53,
        'UFloor'                     : 0.48,
        'UDoor'                      : 2.33,
        'UWindow'                    : 3.49,
        'etaWindowCooling'           : 0.51,  
        'etaWindowHeating'           : 0.51,
        'FlagfWinowInput'            : 1,
        'fWindowCooling'             : 0.70,
        'fWindowHeating'             : 0.60,
        'FlagPerimeterInput'         : 1,
        'psiPerimeter'               : 1.80,
        'psiPerimeterDoor'           : 1.80
    }
}

Convert(d)

KeyError: 'U_roof'